### Assignment 2

In [35]:
import pandas as pd
import numpy as np

In [36]:
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder

Creating a database connection to previous schema by providing all the required details.

In [37]:
host = r'127.0.0.1' 
db = r'MSDS610' 
user = r'postgres' 
pw = r'smooth' 
port = r'5432'
schema = r'raw'

db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [38]:
table_name = r'EPL Data'
schema = r'raw' # schema were the data was loaded last week.

df = pd.read_sql_table(table_name, db_conn, schema)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12153 entries, 0 to 12152
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           12153 non-null  object 
 1   Season         12153 non-null  object 
 2   HomeTeam       12153 non-null  object 
 3   AwayTeam       12153 non-null  object 
 4   FTH Goals      12153 non-null  int64  
 5   FTA Goals      12153 non-null  int64  
 6   FT Result      12153 non-null  object 
 7   HTH Goals      11229 non-null  float64
 8   HTA Goals      11229 non-null  float64
 9   HT Result      11229 non-null  object 
 10  Referee        9329 non-null   object 
 11  H Shots        9329 non-null   float64
 12  A Shots        9329 non-null   float64
 13  H SOT          9329 non-null   float64
 14  A SOT          9329 non-null   float64
 15  H Fouls        9329 non-null   float64
 16  A Fouls        9329 non-null   float64
 17  H Corners      9329 non-null   float64
 18  A Corn

In [40]:
df.head(10)

,Date,Season,HomeTeam,AwayTeam,FTH Goals,FTA Goals,FT Result,HTH Goals,HTA Goals,HT Result,...,H Fouls,A Fouls,H Corners,A Corners,H Yellow,A Yellow,H Red,A Red,Display_Order,League
0,16/01/2025,2024/25,Ipswich Town,Brighton & Hove Albion,0,2,A,0.0,1.0,A,...,13.0,14.0,1.0,9.0,2.0,2.0,0.0,0.0,20250116,Premier League
1,16/01/2025,2024/25,Man United,Southampton,3,1,H,0.0,1.0,A,...,7.0,10.0,4.0,4.0,1.0,3.0,0.0,0.0,20250116,Premier League
2,15/01/2025,2024/25,Everton,Aston Villa,0,1,A,0.0,0.0,D,...,17.0,10.0,8.0,5.0,2.0,1.0,0.0,0.0,20250115,Premier League
3,15/01/2025,2024/25,Leicester,Crystal Palace,0,2,A,0.0,0.0,D,...,7.0,6.0,4.0,3.0,0.0,0.0,0.0,0.0,20250115,Premier League
4,15/01/2025,2024/25,Newcastle,Wolves,3,0,H,1.0,0.0,H,...,10.0,13.0,4.0,2.0,0.0,2.0,0.0,0.0,20250115,Premier League
5,15/01/2025,2024/25,Arsenal,Tottenham,2,1,H,2.0,1.0,H,...,16.0,9.0,10.0,4.0,3.0,1.0,0.0,0.0,20250115,Premier League
6,14/01/2025,2024/25,Brentford,Man City,2,2,D,0.0,0.0,D,...,4.0,4.0,4.0,5.0,0.0,0.0,0.0,0.0,20250114,Premier League
7,14/01/2025,2024/25,Chelsea,Bournemouth,2,2,D,1.0,0.0,H,...,15.0,16.0,9.0,3.0,2.0,3.0,0.0,0.0,20250114,Premier League
8,14/01/2025,2024/25,West Ham,Fulham,3,2,H,2.0,0.0,H,...,9.0,18.0,0.0,3.0,3.0,3.0,0.0,0.0,20250114,Premier League
9,14/01/2025,2024/25,Nott'm Forest,Liverpool,1,1,D,1.0,0.0,H,...,7.0,10.0,0.0,9.0,2.0,1.0,0.0,0.0,20250114,Premier League


Replace missing values with median for numeric columns

In [41]:
numeric_cols = df.select_dtypes(include=['number']).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())

Encoding categorical variables (one-hot encoding)

In [42]:
# categorical_cols = df.select_dtypes(include=['object']).columns
# df = pd.get_dummies(df, columns=categorical_cols) 

Since one hot encoding created too many columns we will be using label encoding instead

In [44]:
categorical_cols = df.select_dtypes(include=['object']).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le

In [45]:
high_cardinality_cols = [col for col in df.select_dtypes(include=['object']).columns if df[col].nunique() >= 50]
df = df.drop(columns=high_cardinality_cols)

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12153 entries, 0 to 12152
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Date           12153 non-null  int32  
 1   Season         12153 non-null  int32  
 2   HomeTeam       12153 non-null  int32  
 3   AwayTeam       12153 non-null  int32  
 4   FTH Goals      12153 non-null  int64  
 5   FTA Goals      12153 non-null  int64  
 6   FT Result      12153 non-null  int32  
 7   HTH Goals      12153 non-null  float64
 8   HTA Goals      12153 non-null  float64
 9   HT Result      12153 non-null  int32  
 10  Referee        12153 non-null  int32  
 11  H Shots        12153 non-null  float64
 12  A Shots        12153 non-null  float64
 13  H SOT          12153 non-null  float64
 14  A SOT          12153 non-null  float64
 15  H Fouls        12153 non-null  float64
 16  A Fouls        12153 non-null  float64
 17  H Corners      12153 non-null  float64
 18  A Corn

Loading data into a new schema named cleaned that was created earlier

In [50]:
table_name = r'EPL Data Cleaned'
schema = r'cleaned' # schema we created this week.

# df.to_sql(table_name, con=db_conn, if_exists='replace', index=False)
df.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

12153